In [1]:
#You can use the line below to install statsmodels package by deleting #
#pip install statsmodels

In [1]:
from math import log, sqrt, exp
import time
import datetime as dt  
from datetime import datetime           # date objects
import numpy as np                      # array manipulation
import matplotlib.pyplot as plot        # plotting
import pandas as pd                     # data analysis
import pandas_datareader as pdr
from scipy.stats import norm  # normal cdf
import csv 
import statsmodels.formula.api as smf # for linear regression
from ib_insync import *
util.startLoop()

In [2]:
try:
    ib.disconnect()
    time.sleep(5)
except:
    print("no ib connection to disconnect")

# initialize connection to IBKR
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=12)  # IB Trader Workstation

no ib connection to disconnect


<IB connected to 127.0.0.1:7497 clientId=12>

In [3]:
#find the future contract
future_date_string='20221116'
future = Future('VIX',future_date_string,'CFE')
ib.qualifyContracts(future)
[vix_con] = ib.reqContractDetails(future)
vix_con

ContractDetails(contract=Contract(secType='FUT', conId=545798747, symbol='VIX', lastTradeDateOrContractMonth='20221116', multiplier='1000', exchange='CFE', currency='USD', localSymbol='VXX2', tradingClass='VX'), marketName='VX', minTick=0.05, sizeMinTick=1.0, orderTypes='ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASKET,BENCHPX,COND,CONDORDER,DAY,DEACT,DEACTDIS,DEACTEOD,GAT,GTC,GTD,GTT,HID,ICE,IOC,LIT,LMT,LTH,MIT,MKT,MTL,NGCOMB,NONALGO,OCA,OPENCLOSE,PEGBENCH,SCALE,SCALERST,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='CFE', priceMagnifier=1, underConId=13455763, longName='CBOE Volatility Index', contractMonth='202211', industry='Indices', category='Volatility Index', subcategory='*', timeZoneId='US/Central', tradingHours='20221011:1700-20221012:1600;20221012:1700-20221013:1600;20221013:1700-20221014:1600;20221015:CLOSED;20221016:1700-20221017:1600;20221017:1700-20221018:1600', liquidHours='20221012:0830-20221012:1515;20221013:0830-202210

In [4]:
#How to get the price data for the vix future
ib.reqMarketDataType(4)
[vix_ticker]=ib.reqTickers(future)
vix_1=vix_ticker.last
vix_ticker.contract

Future(conId=545798747, symbol='VIX', lastTradeDateOrContractMonth='20221116', multiplier='1000', exchange='CFE', currency='USD', localSymbol='VXX2', tradingClass='VX')

In [8]:
#How to get the price of the VIX index
index = Index('VIX')
ib.qualifyContracts(index)
ib.reqContractDetails(index)
[Ticker]=ib.reqTickers(index)
Ticker
vix_spot=Ticker.last

In [9]:
#construct the signal for backwardation/contango
b_signal=(vix_1/vix_spot)-1
b_signal
#if b_signal is positive, vix term structure is in contango
#if b_signal is negative, vix term structure is in bakwardation

-0.05192878338278939

In [10]:
#remaining business days
today=dt.date.today()
future_date=datetime.strptime(future.lastTradeDateOrContractMonth, "%Y%m%d").date()
days = np.busday_count(today, future_date)
days
#daily roll calculation
daily_roll=(vix_1-vix_spot)/days
daily_roll

-0.07000000000000015

Peer closed connection.


In [151]:
order=()
#first trade:
if b_signal<0 and daily_roll<-0.10: 
    order = MarketOrder('BUY', 5)
elif b_signal>0 and daily_roll>0.10:
    order = MarketOrder('SELL', 5)
order

MarketOrder(action='BUY', totalQuantity=5)

In [152]:
#send order to ibkr
trade = ib.placeOrder(future, order)

In [153]:
positions = [p for p in ib.positions() if p.contract.symbol == "VIX"]
future_pos = [p for p in positions if p.contract.secType == "FUT"]
future_pos

[Position(account='DU6066633', contract=Future(conId=540342682, symbol='VIX', lastTradeDateOrContractMonth='20221019', multiplier='1000', currency='USD', localSymbol='VXV2', tradingClass='VX'), position=5.0, avgCost=29952.38)]

In [154]:
future_pos[0].contract.lastTradeDateOrContractMonth

'20221019'

In [138]:
#exit strategy
for f in future_pos:
    days = np.busday_count(today, datetime.strptime(f.contract.lastTradeDateOrContractMonth, "%Y%m%d").date())
    daily_roll=(vix_1-vix_spot)/days #need to find a way to update the price here
    if f.position>0 and (days<=9 or daily_roll>-0.05):
        order = MarketOrder('SELL', f.position)
        trade = ib.placeOrder(future, order)
    elif f.position<0 and (days<=9 or daily_roll<0.05):
        order = MarketOrder('BUY', abs(f.position))
        trade = ib.placeOrder(future, order) 

In [155]:
## Q4- E-mini and VIX Futures Historical Data
today=dt.date.today()
today_str=today.strftime("%m/%d/%Y")
a_year_ago = today - dt.timedelta(days=365)
a_year_ago_str = a_year_ago.strftime("%m/%d/%Y")
#WSJ database didn't work for futures, I got the data from marketwatch for vx00 and es00
vix_futures_data = pd.read_csv('https://www.marketwatch.com/investing/future/vx00/downloaddatapartial?startdate='+a_year_ago_str+'%2000:00:00&enddate='+today_str+'%2000:00:00&daterange=d30&frequency=p1d&csvdownload=true&downloadpartial=false&newdates=false')
sp500_futures=pd.read_csv("https://www.marketwatch.com/investing/future/es00/downloaddatapartial?startdate="+a_year_ago_str+"%2000:00:00&enddate="+today_str+"%2000:00:00&daterange=d30&frequency=p1d&csvdownload=true&downloadpartial=false&newdates=false")
# I will use open prices to calculate the percentage changes
sample = pd.DataFrame(columns=["Date","sp_mini_change","vix_futures_change"])
sample["Date"]=vix_futures_data["Date"]
sample["sp_mini_change"]=sp500_futures['Open'].str.replace(',', '').astype(float)/sp500_futures['Open'].str.replace(',', '').astype(float).shift(periods=-1)-1
sample["vix_futures_change"]=vix_futures_data['Open']/vix_futures_data['Open'].shift(periods=-1)-1
sample.head(10)
#sample is the df we'll use in the regressions for hedge ratio

,Date,sp_mini_change,vix_futures_change
0,09/29/2022,0.016832,-0.038647
1,09/28/2022,0.000136,0.006483
2,09/27/2022,-0.010053,0.024917
3,09/26/2022,-0.019061,0.082734
4,09/23/2022,-0.004415,-0.021127
5,09/22/2022,-0.021850,0.038012
6,09/21/2022,-0.011280,0.032453
7,09/20/2022,0.008613,-0.025735
8,09/19/2022,-0.003714,-0.012704
9,09/16/2022,-0.017120,0.011009


In [156]:
#will use this chunk for change regression
model=smf.ols("vix_futures_change~sp_mini_change",sample).fit()
# get the model beta
beta=model.params.values[1]
print("optimal hedge ratio is "+str(beta))

optimal hedge ratio is -3.1239085760708107


In [157]:
#find the sp500 mini future
#future_date_string needs to be checked
future_date_string='20221216'
future = Future('ES', future_date_string,'GLOBEX')
ib.qualifyContracts(future)
ib.reqContractDetails(future)[0]
#How to get the price data for the vix future
ib.reqMarketDataType(4)
[Ticker]=ib.reqTickers(future)
Ticker
sp500mini_price=Ticker.last
print('sp500mini price is '+str(sp500mini_price))

sp500mini price is 3679.75


In [159]:
Ticker

Ticker(contract=Future(conId=495512551, symbol='ES', lastTradeDateOrContractMonth='20221216', multiplier='50', exchange='GLOBEX', currency='USD', localSymbol='ESZ2', tradingClass='ES'), time=datetime.datetime(2022, 9, 30, 15, 20, 17, 369273, tzinfo=datetime.timezone.utc), marketDataType=3, bid=3679.5, bidSize=45.0, ask=3679.75, askSize=39.0, last=3679.75, lastSize=1.0, volume=1222160.0, open=3656.5, high=3693.75, low=3626.5, close=3654.25)

In [166]:
#Optimal Hedge amount

hedge_amount=(beta*(vix_1*1000)/(sp500mini_price*(vix_con.minTick*100)))
hedge_amount

-5.085185779105688

In [167]:
if hedge_amount<0:
    hedge_order = MarketOrder('SELL', round(abs(hedge_amount)*abs(future_pos[0].position))) #be careful on that f.position if you have more than one contract
else:
    hedge_order = MarketOrder('BUY',round(abs(hedge_amount)*abs(future_pos[0].position)))
hedge_order

MarketOrder(action='SELL', totalQuantity=25)

Peer closed connection.


In [165]:
#send order to ibkr
trade = ib.placeOrder(future, hedge_order)